In [1]:
import pandas as pd
import numpy as np
import pickle
import sys
import time
import glob
import re
from tqdm.notebook import trange, tqdm
import tweepy
from IPython.display import Image, display
from IPython.core.display import HTML 
import json
import utils

In [1]:
auth.get_authorization_url()

NameError: name 'auth' is not defined

In [7]:
auth.request_token = { 'oauth_token' : 'HESdkAAAAAAA2gC0AAABbr554WA',
                         'oauth_token_secret' : '1696396' }

In [8]:
t = auth.get_access_token('1696396')

In [9]:
t

('430684463-hPPEVIKcDIiLGqxw37XuYRgvXNxIBXHO9NZeYz3o',
 'GZ8K5u3SLjdWvXPHSrgfWfyvpAkfKLr8csFo80UBECZ5h')

In [2]:
auth = tweepy.OAuthHandler('QLbvBAuKCD9GOSMzM4ly3Jueb', '9qX5Suoeuoyccict1tkbCheJC42djHrIu71pL0GYnxw1DEhSSp')
auth.set_access_token('430684463-hPPEVIKcDIiLGqxw37XuYRgvXNxIBXHO9NZeYz3o', 'GZ8K5u3SLjdWvXPHSrgfWfyvpAkfKLr8csFo80UBECZ5h')
api = tweepy.API(auth)

In [3]:

#override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
    
    def __init__(self):
        super(MyStreamListener, self).__init__()
        self.users = []
        
    def on_status(self, status):
        if status.user.statuses_count > 3200:
            self.users.append(status.user)
            print(len(self.users))
            if len(self.users) > 500:
                return False
        return True

In [4]:
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tweepy.RateLimitError:
            time.sleep(15 * 60)
        except tweepy.TweepError as e:
            print(e)
            time.sleep(15 * 60)

In [5]:
def get_all_tweets(screen_name):
    alltweets = []
    new_tweets = api.user_timeline(screen_name=screen_name,count=200)

    #save most recent tweets
    alltweets.extend(new_tweets)
    if len(alltweets) == 0:
        return alltweets
    
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print("getting tweets before %s" % (oldest))

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print("...%s tweets downloaded so far" % (len(alltweets)))
    return alltweets

def get_mention_tweets(username, limit=4000):
    statuses = []
    with tqdm(total=limit) as pbar:
        for status in limit_handled(tweepy.Cursor(api.search, q='@' + username).items()):
            if(len(statuses)) > limit:
                break
            statuses.append(status)
            pbar.update(1)
            
    return statuses

def get_list_members(owner, slug, limit):
    users = []
    with tqdm(total=limit) as pbar:
        for user in limit_handled(tweepy.Cursor(api.list_members, owner, slug).items()):
            if(len(users)) > limit:
                break
            users.append(user)
            pbar.update(1)
            
    return users

In [110]:
target_users_df = pd.read_csv('../data/target_users.csv', index_col=0)

In [63]:
for username, r in tqdm(target_users_df.loc[pd.isna(target_users_df['follower_count'])].iterrows()):
    u = api.get_user(username)
    target_users_df.at[username, 'follower_count'] = u.followers_count
    target_users_df.at[username, 'tweets_count'] = u.statuses_count
    target_users_df.at[username, 'created_date'] = u.created_at
    target_users_df.at[username, 'verified'] = u.verified

In [101]:
users = get_list_members('guardian', 'politics-staff', 39)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: DeprecationWarning: generator 'limit_handled' raised StopIteration


In [103]:
female = []
count = 0
started = False
for u in users:
    print(count)
    print("Name: ", u.name)
    print("Description", u.description)
    display(Image(url=u.profile_image_url))
    if input("Is Female: ") == 'y':
        female.append(u)
    print('---------------------')

0
Name:  Daniel Boffey
Description Brussels Bureau Chief, The Guardian Daniel.boffey@theguardian.com


Is Female: n
---------------------
0
Name:  Jackie Ashley
Description Former President, Lucy Cavendish College, University of Cambridge.  Journalist and broadcaster


Is Female: y
---------------------
0
Name:  Polly Toynbee
Description Polly Toynbee is a Guardian columnist, formerly BBC social affairs editor. Her latest  book, with David Walker, is Dismembered, following on from Cameron's Coup


Is Female: y
---------------------
0
Name:  Haroon Siddique
Description Guardian Senior Reporter. The bald facts. haroon[dot]siddique[at]theguardian[dot]com https://t.co/rrGq777EZV


Is Female: n
---------------------
0
Name:  Patrick Butler
Description Social policy editor, The Guardian. Tabloid journalist.


...

Is Female: n
---------------------
0
Name:  MichaelWhite
Description Elderly Hack and Sage


Is Female: n
---------------------
0
Name:  sheilapulham
Description Managing editor, http://t.co/ikD84WLGxk


Is Female: n
---------------------
0
Name:  Dave Hill
Description Writes about London's politics, development and culture. Used to do that for the Guardian. Now runs the website @OnLondon. Centre-left grandad. Busy.


Is Female: n
---------------------
0
Name:  Simon Jeffery
Description Joint deputy head of international, the Guardian


Is Female: n
---------------------


In [67]:
saved = [f.replace('../data/target_users/', '').replace('.p', '') for f in glob.glob("../data/target_users/*.p")]
to_save = target_users_df.loc[target_users_df['mentions_saved']==False].index.values

In [68]:
len(saved), len(to_save)

(897, 0)

In [66]:
target_users_df['mentions_saved'] = [(i in saved) for i, _ in target_users_df.iterrows()]

In [109]:
to_save = target_users_df.loc[target_users_df['mentions_saved']==False].index.values
for user in tqdm(to_save):
    if user in saved:
        continue
    print(user)
    tweets = get_mention_tweets(user)
    pickle.dump(tweets, open('../data/target_users/' + user + '.p', 'wb'))
    

DanielBoffey



JackieAshley


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: generator 'limit_handled' raised StopIteration



pollytoynbee



Haroon_Siddique



patrickjbutler


Twitter error response: status code = 429



KeyboardInterrupt: 

In [87]:
target_users_df.columns

Index(['gender', 'primary_association', 'category', 'follower_count',
       'tweets_count', 'created_date', 'verified', 'mentions_saved'],
      dtype='object')

In [105]:
d = {}
for u in users:
    gender = 'male'
    if u in female:
        gender = 'female'
    d[u.screen_name] = [
        gender, 'thesun', 'journalist', u.followers_count,
       u.statuses_count, u.created_at, u.verified, False
    ]

In [107]:
target_users_df = target_users_df.append(pd.DataFrame.from_dict(d, columns=target_users_df.columns, orient='index'))

In [108]:
len(target_users_df)

627

In [70]:
target_users_df.to_csv('../data/target_users.csv')

In [26]:
senate_target_users = [
    'SenHydeSmith',
    'SenTinaSmith',
    'gillibrandny',
    'SenJackyRosen',
    'SenCortezMasto',
    'SenKamalaHarris',
    'SenMcSallyAZ',
    'SenJoniErnst',
    'SenatorSinema',
    'SenatorBaldwin',
    'SenatorFischer',
    'SenDuckworth',
    'SenWarren',
    'SenatorHassan',
    'SenFeinstein',
    'PattyMurray',
    'MarshaBlackburn',
    'SenCapito',
    'SenatorCantwell',
    'SenatorShaheen',
    'maziehirono',
    'SenStabenow',
    'SenAmyKlobuchar',
    'SenatorCollins',
    'lisamurkowski'
]

In [8]:
target_users = senate_target_users

In [12]:
target_users = [
    'taffyakner'
]

In [6]:
len(target_users)

98

In [43]:
target_users = [u.screen_name for u in female]

In [44]:
for user in tqdm(target_users):
    if user in saved:
        continue
    print(user)
    tweets = get_mention_tweets(user)
    pickle.dump(tweets, open('../data/target_users/' + user + '.p', 'wb'))

In [7]:
nyt_female_users = pickle.load(open('../data/female_nyt.p', 'rb'))

In [30]:
britebart_female = [
    'RAMansour',
    'AmandaLeeHouse',
    'mskristinawong',
    'francesmartel',
    'DCfilmgirl',
    'adrienneross'
]

britebart_male = [
    'ezradulis',
    'Marshal_Dov',
    'joelpollak',
    'peterschweizer',
    'mboyle1',
    'charliespiering',
    'themightygwinn',
    'brandondarby'
]

In [42]:
nyt_female_usernames = [u.screen_name for u in nyt_female_users]
nyt_male_users = [u for u in users if u.screen_name not in nyt_female_usernames]
pickle.dump(nyt_male_users, open('../data/male_nyt.p', 'wb'))
nyt_male_usernames = [u.screen_name for u in nyt_male_users]
saved = [f.replace('../data/target_users/', '').replace('.p', '') for f in glob.glob("../data/target_users/*.p")]

In [37]:
with open('../data/female_dailymail.p', 'rb') as f:
    fusers = pickle.load(f)

In [38]:
fusernames = [u.screen_name for u in fusers]

In [39]:
[u.screen_name for u in users if u.screen_name not in fusernames]

['lukecabrahams',
 'hugolowell',
 'albrummer',
 'JasonGroves1',
 'paulwally3',
 'KevinQuigley_DM',
 'jeffpowell_Mail',
 'p_ash1',
 'FoyChris',
 'EventMagUK',
 'draper_rob',
 'LeosLondon',
 'simon_cass',
 'MailMetroMedia',
 'ByBenGriffiths',
 'riksharma_',
 'matt_dathan',
 'petescu',
 'mike_anstead',
 'wildmanrugby',
 'SamiMokbel81_DM',
 'JamesTapsfield',
 'tony1hazell',
 'Mike_Dickson_DM',
 'JEAlty',
 'Ian_Ladyman_DM',
 'Matt_Barlow_DM',
 'Lawton_Times',
 'Paul_NewmanDM',
 'Daniel_J_Martin',
 'Graham_Poll',
 'jimconey',
 'Ripley_77',
 'jackwdoyle',
 'R_Rai',
 'richardaeden',
 'jonnythegreek',
 'captheath',
 'ColinGYoungie',
 'DominicKing_DM',
 'copesy',
 'marioledwith',
 'jameschappers',
 'williamhanson',
 'dmartosko',
 'charliesale',
 'AlanRoden',
 'AshleyWGray',
 'pmorganbentley',
 'BazBam',
 'chalkmark',
 'dderbyshire',
 'alexrichman',
 'samcunningham',
 'chris__lawrence',
 'RobWaughMail',
 'tomleonard78',
 'JoTweedy',
 'timsculthorpe',
 'johnestevens',
 'guyadams',
 'markprigg']

In [34]:
for user in tqdm(britebart_male):
    if user in saved:
        continue
    print(user)
    tweets = get_mention_tweets(user)
    pickle.dump(tweets, open('../data/target_users/' + user + '.p', 'wb'))
    

In [71]:
users = get_list_members('TheSun', 'sun-people', 126)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: DeprecationWarning: generator 'limit_handled' raised StopIteration


In [73]:
female = []
count = 0
started = False
for u in users:
    print(count)
    print("Name: ", u.name)
    print("Description", u.description)
    display(Image(url=u.profile_image_url))
    if input("Is Female: ") == 'y':
        female.append(u)
    print('---------------------')

0
Name:  Thea Jacobs
Description | Journalist @TheSun's Bizarre | Formerly Sun on Sunday | Trained @NewsAssociates | FOI Fan | Views my own or the cat’s | Tips to thea.jacobs@the-sun.co.uk |


Is Female: y
---------------------
0
Name:  Ben Nicholls
Description Publicist | Editor-in-Chief of Jam Press | Formerly @thesun @metrouk @MailOnline | Do you have a story to tell? info@jampress.co.uk


Is Female: n
---------------------
0
Name:  Mmills5un
Description Working in social media, surviving on Diet Coke and schadenfreude.


Is Female: n
---------------------
0
Name:  Mike Howard
Description Product and Content


Is Female: n
---------------------
0
Name:  Emily Andrews
Description Writer, commentator, realist. Royal Correspondent for The Sun; all views are my own. Creator & co-host of Royal podcast #OnHeir. Royal commentator & expert.


...

Is Female: n
---------------------
0
Name:  Dan Wootton
Description Executive Editor and Columnist at The Sun. talkRADIO host. Subscribe to my podcast The Dan Wootton Interview. Insta @danwootton


Is Female: n
---------------------
0
Name:  john sturgis
Description veteran tabloid hack and centrist dad on media,  food, wine, nature, books, football, cycling and snark


Is Female: n
---------------------
0
Name:  Leigh Holmwood
Description Publicity Consultant @TheXFactor @BGT @GreatDancerTV. Ex-@TheVoiceUK /Deputy Head of TV @TheSun/@Guardian journo. Opinions mine/RTs not necessarily endorsements


Is Female: n
---------------------
0
Name:  Andy Williams
Description Head of Programme for The Sun, former Editor-in-Chief, Yahoo! UK+IE. Drinker of cider, eater of food. Travel a lot. Arsenal man! All me on here.


Is Female: n
---------------------


In [ ]:
 Steve Corbett

In [86]:
pickle.dump(female, open('../data/female_thesun.p', 'wb'))

In [9]:
users_dict = {}
userIds = pd.read_csv('../data/ggautoblocker-blocklist.csv', header=None)[0].values
np.random.shuffle(userIds)

In [10]:
userIds

array([2897957851, 2834175220, 2768226973, ...,  138861093, 2411880967,
        144853440])

In [19]:
for u in tqdm(userIds):
    if u in users_dict.keys():
        continue
    try:
        res = api.get_user(u)
    except tweepy.RateLimitError as e:
        print(e)
        time.sleep(15 * 60)
    except tweepy.TweepError as e:
        if e.api_code == 63:
            print('Suspended: ', u)
            continue
        else:
            break
    users_dict[res.id] = res

[{'message': 'Rate limit exceeded', 'code': 88}]


KeyboardInterrupt: 

In [20]:
len(users_dict)

5400

In [18]:
pickle.dump(users_dict, open('../data/ggautoblocker-blocklist.p', 'wb'))

In [6]:
users_dict = pickle.load(open('../data/NaziBlocker-blocklist.p', 'rb'))

In [21]:
failed = []

In [24]:
scanned_ids = [int(re.findall(r'\d+', f)[0]) for f in glob.glob("../data/ggautoblocker_user_tweets/*.p")]
for i, u in users_dict.items():
    print(i)
    if i in scanned_ids or i in failed:
        continue
    try:
        tweets = get_all_tweets(u.screen_name)
        pickle.dump(tweets, open('../data/ggautoblocker_user_tweets/' + str(i) + '.p', 'wb'))
    except tweepy.RateLimitError:
        time.sleep(15 * 60)
    except Exception as e:
        print(e)
        failed.append(i)

2897957851
2834175220
2768226973
2818295858
2216584854
98725417
376377771
70254047
1572942984
2855287858
65560586
2382626408
2817149054
34292108
2578138580
373733891
2872823574
2428092385
2898014662
1477935842
2658605881
1141177086
2814926215
410110310
217489561
15612544
2893030518
2988356085
26091252
288217960
55797066
1569421542
2787173606
2530819766
2790919300
37645771
258636180
2887482477
2808970238
1546712348
2740563252
2794401939
2781237630
2955331869
317087623
2824531773
2876784264
2431038853
2368238089
1720622760
1542332864
2825382164
839555382
356493351
272602021
15469786
2795045768
2279505289
2788757714
514098973
2876182731
2857609488
1959421836
626590055
602320270
2731149727
2576049210
1705579975
2525048365
2902074476
2607997413
209839643
366015392
2876509224
615155758
2828917938
592482577
204447420
2615968237
1370566902
2280846457
2847059601
2883968269
2914490999
296374317
35091743
866546576
2899754634
550691160
52651155
1276671109
2310819182
2888869228
1977636613
236231313

2917019097
2194575228
353166784
32622476
2843208628
27882729
232936443
504493538
202364438
14298098
271242259
394234937
2782975050
196350226
59591983
2784656491
310511449
336768651
1571231719
260006058
610226810
305833332
128654515
528736220
2859000168
290769869
1024514401
2167016964
2884539886
2854994347
742764410
22987102
217533546
16414733
2836319615
14068645
16289654
2599737889
2190860905
2841652475
2820726802
2835137038
2893529066
1357992846
2838772399
2826761010
2280090253
2785255490
2813859864
1475762880
436195626
171030984
35069350
31036582
2841583822
2303932135
2815520829
1868582228
2904219554
109749172
1149031225
89391806
2878738737
407253509
2896173182
316471808
52120866
188112927
74832429
2172331477
16155438
329297201
2778889014
2840898088
19211812
2824193843
1867371536
2525872736
2874762196
355303821
2886871777
15272071
2723761986
2887833895
2785657215
2778917035
14865775
2782269553
1258157606
45166511
2754837898
2824998107
25755952
2505582426
2783951664
2771235643
2812908

getting tweets before 1149338099119525887
...399 tweets downloaded so far
getting tweets before 1123780658353123335
...599 tweets downloaded so far
getting tweets before 1103675196681142271
...799 tweets downloaded so far
getting tweets before 1081339624625188869
...998 tweets downloaded so far
getting tweets before 1063498053955579903
...1197 tweets downloaded so far
getting tweets before 1043706655035326463
...1397 tweets downloaded so far
getting tweets before 1029916676391948288
...1596 tweets downloaded so far
getting tweets before 903457637077061632
...1786 tweets downloaded so far
getting tweets before 702055422925082623
...1981 tweets downloaded so far
getting tweets before 172058808821100543
...2164 tweets downloaded so far
getting tweets before 77913964402573312
...2164 tweets downloaded so far
2826897919
1043995519
2188417338
160761208
292570498
27721310
213599224
766755139
2298688470
2921153492
2921033731
2640812835
167009538
30180554
22843542
330732361
2968198143
244823773

getting tweets before 920313333642858495
...384 tweets downloaded so far
getting tweets before 822357046817329151
...552 tweets downloaded so far
getting tweets before 791820272005443583
...727 tweets downloaded so far
getting tweets before 751199872641105920
...896 tweets downloaded so far
getting tweets before 744189683085746176
...1073 tweets downloaded so far
getting tweets before 731230341374054404
...1263 tweets downloaded so far
getting tweets before 720127775089291263
...1445 tweets downloaded so far
getting tweets before 715647669251780607
...1617 tweets downloaded so far
getting tweets before 708739040430972927
...1795 tweets downloaded so far
getting tweets before 704051178284425216
...1967 tweets downloaded so far
getting tweets before 700841533029883903
...2162 tweets downloaded so far
getting tweets before 695424497927598080
...2354 tweets downloaded so far
getting tweets before 690014390670725119
...2547 tweets downloaded so far
getting tweets before 684171493442498559
.

...1942 tweets downloaded so far
getting tweets before 1051661306602774527
...2124 tweets downloaded so far
getting tweets before 1049782854266236928
...2308 tweets downloaded so far
getting tweets before 1048606756698378239
...2497 tweets downloaded so far
getting tweets before 1048020991866064895
...2678 tweets downloaded so far
getting tweets before 1047603149408690176
...2848 tweets downloaded so far
getting tweets before 1046846336249737215
...2848 tweets downloaded so far
2266870136
363988731
165744087
219204598
28917852
2797727929
2891563655
2476351008
1246293296
2840808203
113342649
1075108460
2967378916
2374226594
425462828
2794683217
296352111
1007886776
2841557901
766517186
2780919180
2955551520
1454383021
33449297
385107377
385221126
2236273680
360033709
1726670406
1227952580
2917461331
72921705
2364264916
2940833657
513393164
1960483837
384346094
70353153
2752756375
2822207150
2821069051
2870704299
312117970
2802980269
2833979859
1909378328
46440103
2527470768
2801269189
1

...1165 tweets downloaded so far
getting tweets before 1147187842180714496
...1355 tweets downloaded so far
getting tweets before 1139179059081097216
...1541 tweets downloaded so far
getting tweets before 1131399562999017473
...1728 tweets downloaded so far
getting tweets before 1121197581684695039
...1905 tweets downloaded so far
getting tweets before 1112123687338213378
...2091 tweets downloaded so far
getting tweets before 1102387424502648831
...2274 tweets downloaded so far
getting tweets before 1088491007925280767
...2449 tweets downloaded so far
getting tweets before 1071819019823579135
...2643 tweets downloaded so far
getting tweets before 1058077211465601025
...2825 tweets downloaded so far
getting tweets before 1044364373886726145
...3012 tweets downloaded so far
getting tweets before 1031698088405200895
...3038 tweets downloaded so far
getting tweets before 1030138039769735169
...3038 tweets downloaded so far
330201583
2177344856
2547046428
2889011160
32794204
2826731892
1489

getting tweets before 1152327032010751999
...378 tweets downloaded so far
getting tweets before 1076077710659780607
...576 tweets downloaded so far
getting tweets before 1002372619306487809
...774 tweets downloaded so far
getting tweets before 907382649681010687
...974 tweets downloaded so far
getting tweets before 796262375414439935
...1174 tweets downloaded so far
getting tweets before 660558261293223935
...1369 tweets downloaded so far
getting tweets before 577949002085183487
...1566 tweets downloaded so far
getting tweets before 520421622407528447
...1765 tweets downloaded so far
getting tweets before 505400708372631552
...1963 tweets downloaded so far
getting tweets before 476048365894238207
...2163 tweets downloaded so far
getting tweets before 342986192628879359
...2286 tweets downloaded so far
getting tweets before 209768182259515393
...2286 tweets downloaded so far
197265911
544004979
25204604
2883373338
2901769820
2861102078
265163809
126772485
24805460
597268050
273534481
28

...2365 tweets downloaded so far
getting tweets before 931409042165071871
...2546 tweets downloaded so far
getting tweets before 917321692690739199
...2731 tweets downloaded so far
getting tweets before 884310687568338943
...2894 tweets downloaded so far
getting tweets before 865652570630397951
...2901 tweets downloaded so far
getting tweets before 864864813687721983
...2901 tweets downloaded so far
2908485049
2867001425
2826895431
365783222
266766276
2890526130
281838753
2220121043
10424462
1258249279
2775572472
537645896
113137765
717034014
1228989174
2591631931
2924252880
563189195
2977801354
2789150828
2821062391
2839190284
2758213604
587518378
218206813
1191798355
2817444146
20031359
2839286150
2314317558
316645443
2845302284
2401965265
1500034602
496492260
471487282
94595486
109380627
387057670
371148012
2732171486
238469293
385088612
329531410
182197933
2899756927
500230988
38871465
2751259861
23549130
2890968474
954182131
163796311
2883073265
148167165
1556456528
2899762661
877

1657474765
19217369
14189497
2374426496
29497573
312696314
306882723
574110064
33955727
1636427449
208798621
2518058154
20833440
2590942380
2831076659
2794805484
157440424
2924983507
132004975
2951382144
761062446
17953791
2216502884
2794982570
2267762593
170621102
1109003965
1532186774
2788026720
427787245
2895736293
1733467884
438994889
61512994
243739078
2858749471
2884933092
1488189421
509115995
2793323140
1009593492
547727700
2986443586
2831053181
1029320197
99474096
20664158
2905664821
2861987539
2514446251
2938885619
2756361617
308797860
115227131
2579022115
1545093056
161520383
704805650
16661459
1066848919
217163243
2823844460
2866813191
73312013
400353164
1143434060
258536540
2209598131
2957022521
1303983480
2871234696
2739760104
2230804068
2824635320
596868972
2923294977
2985238273
2752863037
17980384
getting tweets before 1108657096998834176
...400 tweets downloaded so far
getting tweets before 1045151528716308479
...600 tweets downloaded so far
getting tweets before 967375

getting tweets before 1199360545645834239
...398 tweets downloaded so far
getting tweets before 1197853902512775167
...598 tweets downloaded so far
getting tweets before 1196540888035549183
...798 tweets downloaded so far
getting tweets before 1195431245855023103
...996 tweets downloaded so far
getting tweets before 1193853679893467135
...1193 tweets downloaded so far
getting tweets before 1192573115898970111
...1392 tweets downloaded so far
getting tweets before 1189341418541780991
...1591 tweets downloaded so far
getting tweets before 1187353717881950208
...1791 tweets downloaded so far
getting tweets before 1185188093214838783
...1991 tweets downloaded so far
getting tweets before 1183513327693107201
...2185 tweets downloaded so far
getting tweets before 1181895446727479295
...2383 tweets downloaded so far
getting tweets before 1179404714489851903
...2580 tweets downloaded so far
getting tweets before 1175895371102466049
...2777 tweets downloaded so far
getting tweets before 1171949

In [158]:
target_users_df.columns

Index(['gender', 'primary_association', 'category', 'follower_count',
       'tweets_count', 'created_date', 'verified', 'mentions_saved',
       'harassment_tweets_prob', 'harassment_fraction_identity_hate',
       'harassment_fraction_insult', 'harassment_fraction_obscene',
       'harassment_fraction_severe_toxic', 'harassment_fraction_threat',
       'harassment_fraction_toxic', 'harassing_users_identity_hate',
       'harassing_users_insult', 'harassing_users_obscene',
       'harassing_users_severe_toxic', 'harassing_users_threat',
       'harassing_users_toxic',
       'harassment_fraction_final_model_gboost_toxic_comments_identity_hate',
       'harassing_users_final_model_gboost_toxic_comments_identity_hate',
       'harassment_fraction_final_model_gboost_toxic_comments_insult',
       'harassing_users_final_model_gboost_toxic_comments_insult',
       'harassment_fraction_final_model_gboost_toxic_comments_obscene',
       'harassing_users_final_model_gboost_toxic_comments_obsc

In [6]:
target_users_df = pd.read_csv('../data/target_users.csv', index_col=0)
users = target_users_df[[
    'harassing_users_final_model_gboost_toxic_comments_identity_hate',
    'harassing_users_final_model_gboost_toxic_comments_insult', 
    'harassing_users_final_model_gboost_toxic_comments_obscene',
    'harassing_users_final_model_gboost_toxic_comments_severe_toxic', 
    'harassing_users_final_model_gboost_toxic_comments_threat',
    'harassing_users_final_model_gboost_toxic_comments_toxic'
]].values.flatten()

to_save = []
for s in users:
    if s is not '[]':
        
        to_save.extend([u for u in re.sub("\[|\]|\'|\,", '', s).split(' ') if len(u) > 0])
to_save = np.unique(to_save)

In [8]:
prefix = '../data/harassers_from_targetted/'
scanned_users = [f.replace(prefix, '').replace('.p', '') for f in glob.glob(prefix + '*.p')]
failed = []

In [7]:
len(to_save), len(scanned_users)

(1511, 1661)

In [9]:
for u in tqdm(to_save):
    if u in scanned_users or u in failed:
        continue
    try:
        tweets = get_all_tweets(u)
        pickle.dump(tweets, open('../data/harassers_from_targetted/' + u + '.p', 'wb'))
    except tweepy.RateLimitError:
        time.sleep(15 * 60)
    except Exception as e:
        print(e)
        failed.append(u)

[{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Not authorized.
Not authorized.
Not authorized.
Not authorized.
Not authorized.
Not authorized.
Not authorized.
Not authorized.
[{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Not authorized.
Not authorized.
[{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Not authorized.
Not authorized.
[{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Not authorized.
Not authorized.
Not authorized.
Not authorized.
[{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Not authorized.
Not authorized.
Not authorized.
Not authorized.
Not authorized.
[{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Not authorized.
Not authorized.
Not authorized.
[{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Not authorized.
Not authorized.
Not authorized.
Not authorized.
[{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Not authorized.
Not authorized.
Not authorized.
Not auth

In [9]:
failed

['0L_N2mQ_oI1w',
 'BCondor331',
 'BelleResistance',
 'Bgnshtmp',
 'BloomConnell',
 'BookWormMamma',
 'CarlaAnnParks2',
 'Eddiem03457426',
 'FLlovesUSA_007',
 'FortinErnel',
 'Fran60909775',
 'Gary19808285',
 'George668433921',
 'HammerJermaine',
 'HighKingKill',
 'Hokie2004',
 'JPM1967',
 'JackMeo07205277',
 'JanBrenner',
 'Jon_of_Yaxley',
 'JoshuaB92148435',
 'Knockonwood10',
 'ManAmongGods',
 'MaryJoh94445574',
 'MellowTd',
 'More_Lefty_Lies',
 'NickRuso1',
 'NickSmithington',
 'Pager727',
 'Pr1nceOfDenmark',
 'Real_WomanYoung',
 'RedEyeRob1',
 'SethWeinstein7',
 'WildaJanes',
 '_CrystalStubbs_',
 'albCapon3',
 'barclay_tas',
 'beewellbeebrave',
 'cdrrock57',
 'dan_jen1',
 'freespe74458524',
 'iHooT',
 'jabberwocky6605',
 'johnnywalters57',
 'joyjohn13638253',
 'lordofethena',
 'mqx34993962',
 'reALEworld',
 'robin_ewry',
 'sdreemer',
 'theRealSeean',
 'weretweetin']

In [ ]:
to_save.extend

In [132]:
re.sub("\[|\]|\'|\,", '', ).split(' ')

['9449sarlo',
 'JoeKienzle',
 'SandraCoppola',
 'jhiessee',
 'heavenlyblue67',
 'GoldenSpur2',
 'HighKingKill']

In [11]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

In [12]:
myStream.sample(languages=['en'])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [20]:
failed = []

In [ ]:
scanned_ids = [int(re.findall(r'\d+', f)[0]) for f in glob.glob("../data/control_user_tweets2/*.p")]
for u in myStreamListener.users:
    i = u.id
    print(i)
    if i in scanned_ids or i in failed:
        continue
    try:
        tweets = get_all_tweets(u.screen_name)
        pickle.dump(tweets, open('../data/control_user_tweets2/' + str(i) + '.p', 'wb'))
    except tweepy.RateLimitError:
        time.sleep(15 * 60)
    except Exception as e:
        print(e)
        failed.append(i)

1127926712640180224
getting tweets before 1210032872863797247
...400 tweets downloaded so far
getting tweets before 1209492932379774975
...600 tweets downloaded so far
getting tweets before 1209151141822574592
...800 tweets downloaded so far
getting tweets before 1208840242058420223
...1000 tweets downloaded so far
getting tweets before 1208522970366455807
...1200 tweets downloaded so far
getting tweets before 1208088529735225343
...1400 tweets downloaded so far
getting tweets before 1207721178984566789
...1600 tweets downloaded so far
getting tweets before 1207085767589322752
...1800 tweets downloaded so far
getting tweets before 1206721600378290177
...2000 tweets downloaded so far
getting tweets before 1206387322155159552
...2200 tweets downloaded so far
getting tweets before 1205908678815211519
...2400 tweets downloaded so far
getting tweets before 1205584667187011583
...2600 tweets downloaded so far
getting tweets before 1205157677904650247
...2800 tweets downloaded so far
getting 

...1405 tweets downloaded so far
getting tweets before 1043716755313623039
...1571 tweets downloaded so far
getting tweets before 1024428397714460671
...1732 tweets downloaded so far
getting tweets before 1006666683472281599
...1889 tweets downloaded so far
getting tweets before 994620311613014015
...2054 tweets downloaded so far
getting tweets before 956189072250359807
...2203 tweets downloaded so far
getting tweets before 905241075933171712
...2349 tweets downloaded so far
getting tweets before 872985261134815231
...2491 tweets downloaded so far
getting tweets before 843247040062771199
...2639 tweets downloaded so far
getting tweets before 833362756086677503
...2656 tweets downloaded so far
getting tweets before 831829467618279425
...2656 tweets downloaded so far
704342063370731520
getting tweets before 1204045306456944640
...394 tweets downloaded so far
getting tweets before 1193288835716800513
...592 tweets downloaded so far
getting tweets before 1183899294153228294
...790 tweets d

...2855 tweets downloaded so far
getting tweets before 565130798555070464
...3031 tweets downloaded so far
getting tweets before 555827678415454207
...3080 tweets downloaded so far
getting tweets before 555410465111965695
...3080 tweets downloaded so far
1194149504297111553
getting tweets before 1208924977833136127
...398 tweets downloaded so far
getting tweets before 1208258294487429119
...597 tweets downloaded so far
getting tweets before 1207488352792043520
...797 tweets downloaded so far
getting tweets before 1206356049055358984
...995 tweets downloaded so far
getting tweets before 1205622311698743297
...1193 tweets downloaded so far
getting tweets before 1204998135233286149
...1389 tweets downloaded so far
getting tweets before 1204562523917365247
...1582 tweets downloaded so far
getting tweets before 1204170344757641215
...1775 tweets downloaded so far
getting tweets before 1203552516283228159
...1966 tweets downloaded so far
getting tweets before 1202813980827369471
...2162 twee

...995 tweets downloaded so far
getting tweets before 1205937052006653951
...1194 tweets downloaded so far
getting tweets before 1204852998570201087
...1392 tweets downloaded so far
getting tweets before 1204018978567397377
...1588 tweets downloaded so far
getting tweets before 1203022657744183295
...1786 tweets downloaded so far
getting tweets before 1201962180247277577
...1983 tweets downloaded so far
getting tweets before 1200872286078803967
...2181 tweets downloaded so far
getting tweets before 1199532456166273024
...2377 tweets downloaded so far
getting tweets before 1198721034272284675
...2576 tweets downloaded so far
getting tweets before 1197969643023093765
...2773 tweets downloaded so far
getting tweets before 1197204877279158273
...2972 tweets downloaded so far
getting tweets before 1196245980385030144
...3167 tweets downloaded so far
getting tweets before 1195376940968030210
...3187 tweets downloaded so far
getting tweets before 1195289288746979327
...3187 tweets downloaded 

...2535 tweets downloaded so far
getting tweets before 1178692938815045631
...2725 tweets downloaded so far
getting tweets before 1160578790302502911
...2916 tweets downloaded so far
getting tweets before 1152954546282356736
...3110 tweets downloaded so far
getting tweets before 1148608730554097663
...3132 tweets downloaded so far
getting tweets before 1148538832712654847
...3132 tweets downloaded so far
16923506
getting tweets before 1204430895115051008
...400 tweets downloaded so far
getting tweets before 1196290431681802240
...599 tweets downloaded so far
getting tweets before 1190772995498921983
...798 tweets downloaded so far
getting tweets before 1180399830679654399
...998 tweets downloaded so far
getting tweets before 1174683038640476161
...1198 tweets downloaded so far
getting tweets before 1169220822403342335
...1397 tweets downloaded so far
getting tweets before 1163675138426122240
...1597 tweets downloaded so far
getting tweets before 1158157589684916229
...1796 tweets downl

...

...3161 tweets downloaded so far
getting tweets before 1176672036514619391
...3175 tweets downloaded so far
getting tweets before 1176649449378058242
...3175 tweets downloaded so far
2957105051
getting tweets before 1196463982825132032
...382 tweets downloaded so far
getting tweets before 1181988814635622399
...559 tweets downloaded so far
getting tweets before 1171129322310111231
...749 tweets downloaded so far
getting tweets before 1157294312033595394
...924 tweets downloaded so far
getting tweets before 1143404019760750591
...1087 tweets downloaded so far
getting tweets before 1130010760430718975
...1247 tweets downloaded so far
getting tweets before 1117096409403199487
...1409 tweets downloaded so far
getting tweets before 1107855277456535553
...1575 tweets downloaded so far
getting tweets before 1095181105668517887
...1731 tweets downloaded so far
getting tweets before 1080756005363945472
...1884 tweets downloaded so far
getting tweets before 1062146470193840128
...2035 tweets dow

...1199 tweets downloaded so far
getting tweets before 1184200848282476550
...1397 tweets downloaded so far
getting tweets before 1179439566119419903
...1597 tweets downloaded so far
getting tweets before 1173776560442236929
...1797 tweets downloaded so far
getting tweets before 1170019246337941504
...1996 tweets downloaded so far
getting tweets before 1165398481504231425
...2195 tweets downloaded so far
getting tweets before 1161370793206583297
...2394 tweets downloaded so far
getting tweets before 1155987381511606279
...2593 tweets downloaded so far
getting tweets before 1150467611978686463
...2792 tweets downloaded so far
getting tweets before 1144208006990716927
...2991 tweets downloaded so far
getting tweets before 1137881031250522111
...3190 tweets downloaded so far
getting tweets before 1132012901613670401
...3237 tweets downloaded so far
getting tweets before 1129772333114253317
...3237 tweets downloaded so far
986661813445017601
getting tweets before 1208235674526273535
...399

...2324 tweets downloaded so far
getting tweets before 1191888085589667840
...2515 tweets downloaded so far
getting tweets before 1190250827782311936
...2706 tweets downloaded so far
getting tweets before 1188868884021071877
...2898 tweets downloaded so far
getting tweets before 1187742744304857087
...3089 tweets downloaded so far
getting tweets before 1184929156741468159
...3123 tweets downloaded so far
getting tweets before 1184847589822230527
...3123 tweets downloaded so far
3535226893
getting tweets before 1205170293720768516
...395 tweets downloaded so far
getting tweets before 1199758549921886207
...589 tweets downloaded so far
getting tweets before 1194645529281495040
...784 tweets downloaded so far
getting tweets before 1192496916480741375
...979 tweets downloaded so far
getting tweets before 1190933386518388735
...1172 tweets downloaded so far
getting tweets before 1190328807988686848
...1366 tweets downloaded so far
getting tweets before 1186904779735191552
...1552 tweets dow

...398 tweets downloaded so far
getting tweets before 1204237263695073279
...595 tweets downloaded so far
getting tweets before 1202480111007617025
...792 tweets downloaded so far
getting tweets before 1200972149361496063
...988 tweets downloaded so far
getting tweets before 1199786656028512256
...1186 tweets downloaded so far
getting tweets before 1198658811273170943
...1386 tweets downloaded so far
getting tweets before 1196985032813862911
...1586 tweets downloaded so far
getting tweets before 1195865866954407936
...1777 tweets downloaded so far
getting tweets before 1193043241597169663
...1974 tweets downloaded so far
getting tweets before 1191545947106369535
...2172 tweets downloaded so far
getting tweets before 1190410974286663682
...2371 tweets downloaded so far
getting tweets before 1189019505235615743
...2569 tweets downloaded so far
getting tweets before 1187214874041913344
...2769 tweets downloaded so far
getting tweets before 1185243239659528198
...2966 tweets downloaded so 

...1708 tweets downloaded so far
getting tweets before 1176397332272861183
...1892 tweets downloaded so far
getting tweets before 1172030879700529151
...2074 tweets downloaded so far
getting tweets before 1168939606361985028
...2256 tweets downloaded so far
getting tweets before 1165947475544420351
...2447 tweets downloaded so far
getting tweets before 1161108121504374783
...2629 tweets downloaded so far
getting tweets before 1158221313124814850
...2801 tweets downloaded so far
getting tweets before 1152991237433176069
...2976 tweets downloaded so far
getting tweets before 1148549199937585151
...2979 tweets downloaded so far
getting tweets before 1148362289118162943
...2979 tweets downloaded so far
3112314314
getting tweets before 1108728996072701951
...341 tweets downloaded so far
getting tweets before 1017507198262489087
...490 tweets downloaded so far
getting tweets before 955254128430800895
...583 tweets downloaded so far
getting tweets before 905855332257320963
...698 tweets downl

In [ ]:
failed